In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import linalg

import matplotlib.pyplot as plt

In [ ]:
tickers = yf.Tickers('msft aapl goog')

df_msft = tickers.tickers['MSFT'].history(period="1d", interval="1m")["Close"]
df_aapl = tickers.tickers['AAPL'].history(period="1d", interval="1m")["Close"]
df_goog = tickers.tickers['GOOG'].history(period="1d", interval="1m")["Close"]

In [ ]:
df_test = tickers.tickers['MSFT'].history(period="1d", interval="1m")

In [ ]:
print(df_test)

In [ ]:
# Creating a dictionary by passing Series objects as values
frame = {'MSFT': df_msft},
        'GOOG': df_goog,
        'AAPL': df_aapl}
 
# Creating DataFrame by passing Dictionary
df = pd.DataFrame(frame)

In [ ]:
display(df)

In [ ]:
lrets = np.log(df/df.shift(1)).dropna()

In [ ]:
Omega = (lrets
         .rolling(30)
         .cov()
         .dropna())

In [ ]:
display(Omega)

In [ ]:
dates = lrets.index
dates = dates[30:]

covdata = dict(zip(dates, [Omega.loc[date].values for date in dates]))

In [ ]:
marketBM = dict(zip(dates, [np.matmul(np.linalg.inv(linalg.sqrtm(Omega.loc[date].values)),lrets.loc[date].values) for date in dates]))

In [ ]:
plt.plot([marketBM[date][0] for date in dates])
plt.show()
print(np.std(np.array([marketBM[date][0] for date in dates])))
print(np.mean(np.array([marketBM[date][0] for date in dates])))

In [ ]:
K=3
BMmarketpath = 1/250*np.ones((len(dates),K))
mean=np.zeros(K)
for k in range(K):
    mean[k] = np.mean([marketBM[l][k] for l in dates])

In [ ]:
for i in range(K):
    BMmarketpath[:,i] = np.cumsum([np.sqrt(1/250)*(marketBM[date][i]-0.*mean[i])
                                                   for date in dates])
#BMmarketpath[:,K]= np.cumsum(BMmarketpath[:,K])
plt.plot(BMmarketpath)
plt.show()

In [ ]:
plt.figure()
plt.plot(df_msft)
plt.plot(df_aapl)
plt.plot(df_goog)
plt.show()

For example, the New York Stock Exchange is, as of 2020, open from 9:30 AM Eastern Time to 4:00 PM Eastern Time.

In [ ]:
print(df_msft.shape)

In [ ]:
plt.figure()
plt.plot(df_msft[:700])
plt.show()

In [ ]:
Kmarket = 3
Ymarket = np.zeros((len(dates),Kmarket))
for i in range(Kmarket):
    Ymarket[:,i]=np.cumsum([lrets.loc[date].values[i] for date in dates])
Ymarket = np.exp(Ymarket)

In [ ]:
plt.plot(Ymarket)
plt.show()

In [ ]:
print(Ymarket.shape)

In [ ]:
d=K
M=70

def nilpotent(M):
    B = np.zeros((M,M))
    for i in range(2,M):
        B[i,i-1]=1.0
    return B

def canonical(i,M):
    e = np.zeros((M,1))
    e[i,0]=1.0
    return e

def randomAbeta(d,M):
    A = []
    beta = []
    for i in range(d):
        #B = 0.1*np.identity(M)+np.random.normal(0.0,.5,size=(M,M))
        B = np.random.normal(0.0,0.02,size=(M,M)) # 0.1 for scen-gen, 1.5 for SABR
        #B = np.random.permutation(B)
        #B = np.identity(M)
        #B = sp.linalg.sqrtm(np.matmul(B,np.transpose(B)))
        A = A + [B]
        beta = beta + [0.*canonical(i,M)+np.random.normal(0.0,0.03,size=(M,1))]
    return [A,beta]

Abeta = randomAbeta(d,M)
A = Abeta[0]
beta = Abeta[1]

def sigmoid(x):
    return np.tanh(x)

def reservoirfield(state,increment):
    value = np.zeros((M,1))
    for i in range(d):
        value = value + sigmoid(np.matmul(A[i],state) + beta[i])*increment[i]
    return value

In [ ]:
def vectorfield2d(state,increment):
    return np.array([(2.0*np.sqrt(state[1]**2))**0.7+np.sin(state[1]),1.0*state[1]+np.cos(state[1])])*increment[0]+np.array([(2.0*np.sqrt(state[1]**2))**0.7,0.0*state[1]])*increment[1]

class SDE:
    def __init__(self,timehorizon,initialvalue,dimension,dimensionBM,dimensionR,vectorfield,timesteps):
        self.timehorizon = timehorizon
        self.initialvalue = initialvalue # np array
        self.dimension = dimension
        self.dimensionBM = dimensionBM
        self.dimensionR = dimensionR
        self.vectorfield = vectorfield
        self.timesteps = timesteps

    def path(self):
        BMpath = [np.zeros(self.dimensionBM)] # We store it in a list to have the whole BM path
        SDEpath = [np.array([1.0, self.initialvalue])]
        for i in range(self.timesteps):
            helper = np.random.normal(0,np.sqrt(self.timehorizon/self.timesteps),self.dimensionBM)
            BMpath = BMpath + [BMpath[-1]+helper]
            SDEpath = SDEpath + [np.exp(-1.0*self.timehorizon/self.timesteps)*(SDEpath[-1]+self.vectorfield(SDEpath[-1],helper))] # represents the path of X_t using the SDE (and hence the vf)
        return [BMpath, SDEpath]

    def anypath(self):
        BMpath = [np.zeros(self.dimensionBM)]
        SDEpath = [np.array([1.0, self.initialvalue])]#[np.ones((self.dimension,1))*self.initialvalue]

        for i in range(self.timesteps):
            helper = np.cos(BMpath[-1]*50)*self.timehorizon/self.timesteps#np.random.normal(0,np.sqrt(self.timehorizon/self.timesteps),self.dimensionBM)
            BMpath = BMpath + [BMpath[-1]+helper]
            SDEpath = SDEpath + [np.exp(-0.0*self.timehorizon/self.timesteps)*(SDEpath[-1]+self.vectorfield(SDEpath[-1],helper))]

        return [BMpath, SDEpath]

    def reservoir(self,BMpath,scaling,k):
        reservoirpath = [canonical(k,self.dimensionR)*self.initialvalue] # e_k*X_0
        for i in range(self.timesteps):
            increment = scaling*(BMpath[i+1]-BMpath[i])
            reservoirpath = reservoirpath + [np.exp(-1.0*self.timehorizon/self.timesteps)*(reservoirpath[-1]+reservoirfield(reservoirpath[-1],increment))]
        return reservoirpath

In [ ]:
#Ymarketdiff = np.diff(Ymarket,axis=0)
#Ymarkettrain = np.concatenate((Ymarket[:200],Ymarketdiff[:200:1]),axis=0)
Ymarkettrain = Ymarket[:300]
np.shape(Ymarkettrain)

In [ ]:
BMmarketpathlist = [BMmarketpath[i,:] for i in range(len(dates))]
#BMmarketpathlist = [Ymarket[i,:K] for i in range(len(dates))]
Reservoir = SDE(1,1.5,2,K,M,vectorfield2d,len(dates)-1)
X=Reservoir.reservoir(BMmarketpathlist,0.5,0)
np.shape(X)
Xdata = np.squeeze(X)
Reservoir1 = SDE(1,0.0,2,K,M,vectorfield2d,len(dates)-1)
X1=Reservoir1.reservoir(BMmarketpathlist,0.4,0)
np.shape(X1)
Xdata1=np.squeeze(X1)
Reservoir2 = SDE(1,-1.5,2,K,M,vectorfield2d,len(dates)-1)
X2=Reservoir2.reservoir(BMmarketpathlist,0.3,0)
np.shape(X2)
Xdata2 = np.squeeze(X2)
Xdata = np.concatenate((Xdata,Xdata1,Xdata2),axis=1)
#Xdatadiff = np.diff(Xdata,axis=0)
#Xtrain=np.concatenate((Xdata[:200],Xdatadiff[:200:1]),axis=0)
Xtrain = Xdata[:300]
np.shape(Xtrain)

In [ ]:
from sklearn import linear_model
lm = linear_model.Ridge(alpha=0.05)#LinearRegression()
model = lm.fit(Xtrain,Ymarkettrain)
plt.plot(model.predict(Xtrain),'r')
plt.plot(Ymarkettrain,'b')
plt.show()
model.score(Xtrain,Ymarkettrain)
model.coef_

In [ ]:
f,p=plt.subplots(Kmarket,1,figsize=(6,6),sharey=False)

for i in range(Kmarket):
    p[i].plot(model.predict(Xdata[301:])[:,i],'b')
    p[i].plot(Ymarket[301:][:,i],'g')
plt.show()

Refine these inputs to have more data